In [1]:
# *Väestöennusteisiin tarvittavien Tilastokeskuskoodien ajo ja haku;
# Haetaan Tilastokeskukselta kuntien välinen muutto - tarvitaan koodissa 3_SEUDUNENN
# 2025 Juuso Heinisuo - Kangasalan kaupunki
# https://github.com/kangasalakehitys/vaestoennuste

from pyjstat import pyjstat
import requests
from functools import reduce
import pandas as pd
import numpy as np
import openpyxl
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Kirjautumistunnukset
un = 'XXXXX'
pw = 'XXXX'

#Exceliin ennustevuodet
file_exc = 'Ulkomaat_lähtömuutto_Ve1.xlsx'
ENNUSTEVUODET = ["2025","2026","2027","2028","2029","2030","2031","2032","2033","2034","2035"]
ALUE = ["1","1","1","1","1","1","1","1","1","1","1"]
TYYPPI = ["1","1","1","1","1","1","1","1","1","1","1"]

# Tilastokeskuksen kyselyyn parametrit
OMA_KUNTA = ["KU211"]
IKÄRYHMÄT = ["SSS","0-4","5-9","10-14","15-19","20-24","25-29","30-34","35-39","40-44","45-49","50-54","55-59","60-64","65-69","70-74","75-"]
VUODET = ["2020","2021","2022","2023","2024"]

# Valitaan oma maakunta
OMA_MAAKUNTA = '06'

# Rajapinnan osoite 12pr -- Maahan- ja maastamuutto muuttaneiden iän ja kielen mukaan kunnittain, 2010-2024
POST_URL = 'https://pxhopea2.stat.fi/PXWeb/api/v1/fi/Muuttoliike_ja_vaestonmuutokset/Muuttoliike_ja_vaestonmuutokset_aikasarja/Muuttoliike/muvamuu_12pr.px'
headers = {'Content-Type': 'application/json', 'un': un, 'pw': pw }

In [3]:
query = {
  "query": [
    {
      "code": "Kunta",
      "selection": {
        "filter": "item",
        "values": OMA_KUNTA
      }
    },
    {
      "code": "Ikä",
      "selection": {
        "filter": "item",
        "values": IKÄRYHMÄT
      }
    },
    {
      "code": "Kieli",
      "selection": {
        "filter": "item",
        "values": [
          "SSS"
        ]
      }
    },
    {
      "code": "Vuosi",
      "selection": {
        "filter": "item",
        "values": VUODET
      }
    },
    {
      "code": "Tiedot",
      "selection": {
        "filter": "item",
        "values": [
          "Maastamuutto"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}
resultat = requests.post(POST_URL, headers=headers, json = query)
print(resultat)


<Response [200]>


In [4]:
dataset = pyjstat.Dataset.read(resultat.text)

In [5]:
#Write dataframe
df = dataset.write('dataframe')
#Get all values
df_all = df[(df['Ikä'].str.contains('Yhteensä'))]
#Get men
df_men = df[(df['Ikä'].apply(lambda x: 'Yhteensä' not in x))]
#Get women
df_women = df[(df['Ikä'].apply(lambda x: 'Yhteensä' not in x))]

df.head(200)


,Kunta,Ikä,Kieli,Vuosi,Tiedot,value
0,Kangasala,Yhteensä,Yhteensä,2020,Maastamuutto Suomesta,27
1,Kangasala,Yhteensä,Yhteensä,2021,Maastamuutto Suomesta,39
2,Kangasala,Yhteensä,Yhteensä,2022,Maastamuutto Suomesta,52
3,Kangasala,Yhteensä,Yhteensä,2023,Maastamuutto Suomesta,36
4,Kangasala,Yhteensä,Yhteensä,2024,Maastamuutto Suomesta,42
...,...,...,...,...,...,...
80,Kangasala,75 -,Yhteensä,2020,Maastamuutto Suomesta,0
81,Kangasala,75 -,Yhteensä,2021,Maastamuutto Suomesta,0
82,Kangasala,75 -,Yhteensä,2022,Maastamuutto Suomesta,1
83,Kangasala,75 -,Yhteensä,2023,Maastamuutto Suomesta,0


In [6]:
#Year average
df_group_all = round(df_all.groupby(['Kunta']).agg(yht=('value', 'mean')),0)
#Year average men
df_group_men = pd.DataFrame(df_men.groupby(['Kunta', 'Ikä']).agg(m=('value', 'mean')))
#Year average women
df_group_women = pd.DataFrame(df_women.groupby(['Kunta', 'Ikä']).agg(n=('value', 'mean')))

# Convert groupby object to DataFrame 
df_group_all_1 = df_group_all.reset_index()
df_group_men_1 = df_group_men.reset_index()
df_group_women_1 = df_group_women.reset_index()

df_group_women_1.drop(columns=["Ikä"], inplace=True, errors='ignore')

# Repeat value
df_group_all_1 = pd.DataFrame(np.repeat(df_group_all_1.values, repeats=len(df_group_men_1.index), axis=0), columns=df_group_all_1.columns)

df_group = pd.concat([df_group_men_1,df_group_women_1,df_group_all_1], axis=1)
columns_to_drop = ["Kunta"]
df_group.drop(columns=columns_to_drop, inplace=True, errors='ignore')

#Sukupuolien osuudet muutoissa
df_group['m_pro'] = (df_group['m'] / df_group['yht'])
df_group['n_pro'] = (df_group['n'] / df_group['yht'])

#Jaetaan 5, jotta saadaan 1-vuodelle luvut
df_group['m_pro'] = (df_group['m_pro'] / 5)
df_group['n_pro'] = (df_group['n_pro'] / 5)

print(df_group)


        Ikä    m    n   yht     m_pro     n_pro
0     0 - 4  3.4  3.4  39.0  0.017436  0.017436
1   10 - 14  1.4  1.4  39.0  0.007179  0.007179
2   15 - 19  2.0  2.0  39.0  0.010256  0.010256
3   20 - 24  3.8  3.8  39.0  0.019487  0.019487
4   25 - 29  6.0  6.0  39.0  0.030769  0.030769
5   30 - 34  4.8  4.8  39.0  0.024615  0.024615
6   35 - 39  4.0  4.0  39.0  0.020513  0.020513
7   40 - 44  2.2  2.2  39.0  0.011282  0.011282
8   45 - 49  3.0  3.0  39.0  0.015385  0.015385
9     5 - 9  2.2  2.2  39.0  0.011282  0.011282
10  50 - 54  2.0  2.0  39.0  0.010256  0.010256
11  55 - 59  1.4  1.4  39.0  0.007179  0.007179
12  60 - 64  1.2  1.2  39.0  0.006154  0.006154
13  65 - 69  0.8  0.8  39.0  0.004103  0.004103
14  70 - 74  0.4  0.4  39.0  0.002051  0.002051
15     75 -  0.6  0.6  39.0  0.003077  0.003077


In [7]:
#Generate years
ikä = [x for x in range(100)]
df_ikä = pd.DataFrame(ikä, columns=['Ikä']).apply(pd.to_numeric)

conditions = [
    (df_ikä['Ikä'] <= 4),
    (df_ikä['Ikä'] > 4) & (df_ikä['Ikä'] <= 9),
    (df_ikä['Ikä'] > 9) & (df_ikä['Ikä'] <= 14),
    (df_ikä['Ikä'] > 14) & (df_ikä['Ikä'] <= 19),
    (df_ikä['Ikä'] > 19) & (df_ikä['Ikä'] <= 24),
    (df_ikä['Ikä'] > 24) & (df_ikä['Ikä'] <= 29),
    (df_ikä['Ikä'] > 29) & (df_ikä['Ikä'] <= 34),
    (df_ikä['Ikä'] > 34) & (df_ikä['Ikä'] <= 39),
    (df_ikä['Ikä'] > 39) & (df_ikä['Ikä'] <= 44),
    (df_ikä['Ikä'] > 44) & (df_ikä['Ikä'] <= 49),
    (df_ikä['Ikä'] > 49) & (df_ikä['Ikä'] <= 54),
    (df_ikä['Ikä'] > 54) & (df_ikä['Ikä'] <= 59),
    (df_ikä['Ikä'] > 59) & (df_ikä['Ikä'] <= 64),
    (df_ikä['Ikä'] > 64) & (df_ikä['Ikä'] <= 69),
    (df_ikä['Ikä'] > 69) & (df_ikä['Ikä'] <= 74),
    (df_ikä['Ikä'] > 74)
]

# create a list of the values we want to assign for each condition
values = ['0 - 4', '5 - 9', '10 - 14', '15 - 19', '20 - 24', '25 - 29', '30 - 34', 
          '35 - 39', '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64', '65 - 69', '70 - 74', '75 -']

df_ikä['Ikä_lk'] = np.select(conditions, values, default='Other')

# Merging on multiple columns
res = pd.merge(df_ikä, df_group, left_on=['Ikä_lk'], right_on=['Ikä'], how='inner')

# *Generoidaan sarakenimet;
men = pd.DataFrame(['mul' + str(x) for x in range(100)])
women = pd.DataFrame(['nul' + str(x) for x in range(100)])

res['men'] = men
res['women'] = women



print(res)



    Ikä_x Ikä_lk  Ikä_y    m    n   yht     m_pro     n_pro    men  women
0       0  0 - 4  0 - 4  3.4  3.4  39.0  0.017436  0.017436   mul0   nul0
1       1  0 - 4  0 - 4  3.4  3.4  39.0  0.017436  0.017436   mul1   nul1
2       2  0 - 4  0 - 4  3.4  3.4  39.0  0.017436  0.017436   mul2   nul2
3       3  0 - 4  0 - 4  3.4  3.4  39.0  0.017436  0.017436   mul3   nul3
4       4  0 - 4  0 - 4  3.4  3.4  39.0  0.017436  0.017436   mul4   nul4
..    ...    ...    ...  ...  ...   ...       ...       ...    ...    ...
95     95   75 -   75 -  0.6  0.6  39.0  0.003077  0.003077  mul95  nul95
96     96   75 -   75 -  0.6  0.6  39.0  0.003077  0.003077  mul96  nul96
97     97   75 -   75 -  0.6  0.6  39.0  0.003077  0.003077  mul97  nul97
98     98   75 -   75 -  0.6  0.6  39.0  0.003077  0.003077  mul98  nul98
99     99   75 -   75 -  0.6  0.6  39.0  0.003077  0.003077  mul99  nul99

[100 rows x 10 columns]


In [8]:
#Create final Excelformat
df_out_men = res[["m_pro", "men"]].copy()
df_out_men = df_out_men.set_index('men').transpose()

df_out_women = res[["n_pro", "women"]].copy()
df_out_women = df_out_women.set_index('women').transpose()

df_out_all = res[["yht"]].copy()
df_out_all_first = df_out_all.iloc[:1]
df_out_all_first.rename(columns={'yht': 'umlyht'}, inplace=True)

list_of_years = list(zip(ENNUSTEVUODET))
list_of_years

df_vuodet = pd.DataFrame(list_of_years,
                  columns=['vuosi'])

df_out_tyyppi = pd.DataFrame(np.array([['1']]))
df_out_tyyppi.columns = ['tyyppi']
df_vuodet.columns = ['vuosi']
df_out_alue = pd.DataFrame(np.array([['1']]))
df_out_alue.columns = ['alue']



final = pd.concat([
    df_out_alue.reset_index(drop=True),
    df_out_tyyppi.reset_index(drop=True),
    df_out_all_first.reset_index(drop=True),
    df_out_men.reset_index(drop=True),
    df_out_women.reset_index(drop=True)], axis=1)

# Repeat value
final2 = pd.DataFrame(np.repeat(final.values, repeats=len(df_vuodet), axis=0), columns=final.columns)

final3 = pd.concat([df_vuodet, final2], axis=1)

print(final3)

final3.to_excel(file_exc, sheet_name='Sheet_name_1', index=False)



   vuosi alue tyyppi umlyht      mul0      mul1      mul2      mul3      mul4  \
0   2025    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   
1   2026    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   
2   2027    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   
3   2028    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   
4   2029    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   
5   2030    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   
6   2031    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   
7   2032    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   
8   2033    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   
9   2034    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   
10  2035    1      1   39.0  0.017436  0.017436  0.017436  0.017436  0.017436   

        mul5  ...     nul90